In [1]:
import mcdm
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.methods import VIKOR
from pymcdm.methods import PROMETHEE_II
from pymcdm.helpers import rankdata,rrankdata
from pymcdm import weights as mcdm_weights

In [2]:
from prettytable import from_csv
with open("liver_perform.tsv") as fp:
    mytable = from_csv(fp)
print(mytable)

+----------------------+----------+-----------+--------+-------------+---------+-------------+-------------+-------------+--------------+--------------+
|                      | ACCURACY | PRECISION | RECALL |     AUC     | F1SCORE |   FP_RATE   |   FN_RATE   | SPECIFICITY | HAMMING LOSS | ENTROPY LOSS |
+----------------------+----------+-----------+--------+-------------+---------+-------------+-------------+-------------+--------------+--------------+
| LOGISTIC REGRESSION  |   0.7    |    0.65   |  0.7   | 0.671238245 |   0.67  | 0.827586207 | 0.136363636 | 0.172413793 | 0.299145299  | 10.33227662  |
|    DECISION TREE     |   0.61   |    0.62   |  0.61  | 0.484326019 |   0.61  |  0.75862069 | 0.272727273 |  0.24137931 | 0.393162393  | 13.57949834  |
| GAUSSIAN NAIVE BAYES |   0.5    |    0.83   |  0.5   | 0.684169279 |   0.5   |      0      | 0.670454545 |      1      | 0.504273504  | 17.41698981  |
|         SVM          |   0.62   |    0.59   |  0.62  | 0.587774295 |   0.6   | 0

In [10]:
x_matrix, alt_names = mcdm.load("liver_perform.tsv", delimiter="\t", skiprows=1, labeled_rows=True)
alts = np.array(x_matrix, dtype='float')
weights = mcdm_weights.equal(alts)
types = np.array([1,1,1,1,1,-1,-1,1,-1,-1])

In [11]:
topsis = TOPSIS()
# Determine preferences and ranking for alternatives
pref1 = topsis(alts, weights, types)
ranking1= rrankdata(pref1)

for r, p in zip(ranking1, pref1):
    print(r, p)


2.0 0.6249495433030752
6.0 0.4671348796103944
4.0 0.5549153732254491
5.0 0.48697852796240343
3.0 0.5568334656413534
1.0 0.7369595509683988
7.0 0.44893601553232615


In [12]:
vikor= VIKOR()
pref2 = vikor(alts, weights, types)
ranking2= rankdata(pref2)

for r, p in zip(ranking2, pref2):
    print(r, p)

2.0 0.4667951579547199
6.0 0.9290688763506856
4.0 0.7605387303353439
5.0 0.8209821833219335
3.0 0.731188484272285
1.0 0.0
7.0 1.0


In [13]:
promethe=PROMETHEE_II('usual')
p = np.random.rand(x_matrix.shape[1]) / 2
q = np.random.rand(x_matrix.shape[1]) / 2 + 0.5
pref3 = promethe(alts, weights, types,p=p,q=q)
ranking3 = rrankdata(pref3)

for r, p in zip(ranking3, pref3):
    print(r, p)

2.0 0.36666666666666653
7.0 -0.3333333333333333
4.0 -0.09999999999999998
5.0 -0.23333333333333323
3.0 0.1333333333333333
1.0 0.43333333333333335
6.0 -0.26666666666666666


In [14]:
from prettytable import PrettyTable
print("#########################LIVER DATASET#######################")
columns = ["Model", "TOPSIS","Value","VIKOR","VALUES","PROMETHEE","VALUES"]
Table = PrettyTable()
Table.add_column(columns[0], ["LOGISTIC REGRESSION ", "DECISION TREE", "GAUSSIAN NAIVE BAYES",
                       "SVM ", "NEURAL NETWORK", "RANDOM FOREST ", "LINEAR MODEL"])
Table.add_column(columns[1], ranking1)
Table.add_column(columns[2], pref1)
Table.add_column(columns[3], ranking2)
Table.add_column(columns[4], pref2)
Table.add_column(columns[5], ranking3)
Table.add_column(columns[6], pref3)

print(Table)

#########################LIVER DATASET#######################
+----------------------+--------+---------------------+-------+--------------------+-----------+----------------------+
|        Model         | TOPSIS |        Value        | VIKOR |       VALUES       | PROMETHEE |        VALUES        |
+----------------------+--------+---------------------+-------+--------------------+-----------+----------------------+
| LOGISTIC REGRESSION  |  2.0   |  0.6249495433030752 |  2.0  | 0.4667951579547199 |    2.0    | 0.36666666666666653  |
|    DECISION TREE     |  6.0   |  0.4671348796103944 |  6.0  | 0.9290688763506856 |    7.0    | -0.3333333333333333  |
| GAUSSIAN NAIVE BAYES |  4.0   |  0.5549153732254491 |  4.0  | 0.7605387303353439 |    4.0    | -0.09999999999999998 |
|         SVM          |  5.0   | 0.48697852796240343 |  5.0  | 0.8209821833219335 |    5.0    | -0.23333333333333323 |
|    NEURAL NETWORK    |  3.0   |  0.5568334656413534 |  3.0  | 0.731188484272285  |    3.0    |  

In [15]:
ans=mcdm.rank(x_matrix, alt_names=alt_names, n_method="Linear2", w_method="CRITIC", s_method="SAW") 
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################SAW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table) 

#########################SAW#######################
+----------------------+---------------------+
|        Model         |        Value        |
+----------------------+---------------------+
|    RANDOM FOREST     |  0.6008123925655068 |
| LOGISTIC REGRESSION  |  0.5537581408276082 |
|     LINEAR MODEL     |  0.5487894498073074 |
| GAUSSIAN NAIVE BAYES |  0.5402210303349484 |
|         SVM          | 0.46610818194815995 |
|    NEURAL NETWORK    |  0.4596305755791673 |
|    DECISION TREE     |  0.4338378982914595 |
+----------------------+---------------------+


In [16]:
ans=mcdm.rank(x_matrix,alt_names=alt_names,n_method="Linear1",w_method="CRITIC",s_method="MEW")
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################MEW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table)

#########################MEW#######################
+----------------------+--------------------+
|        Model         |       Value        |
+----------------------+--------------------+
|    RANDOM FOREST     | 0.5639249831479368 |
|    DECISION TREE     | 0.5485920508629925 |
|         SVM          | 0.5088748941820731 |
| LOGISTIC REGRESSION  | 0.4856761739825573 |
| GAUSSIAN NAIVE BAYES |        0.0         |
|    NEURAL NETWORK    |        0.0         |
|     LINEAR MODEL     |        0.0         |
+----------------------+--------------------+
